In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [4]:
# ====== Device ======
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# ====== Hyperparameters ======
image_size = 28
batch_size = 32
epochs = 15
learning_rate = 0.001

Using device: cuda


In [5]:
# Training transforms (data augmentation)
train_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((image_size, image_size)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1,0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

val_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Datasets
train_dataset = datasets.ImageFolder('/kaggle/input/handwritten-dataset/train', transform=train_transform)
val_dataset = datasets.ImageFolder('/kaggle/input/handwritten-dataset/val', transform=val_transform)
test_dataset = datasets.ImageFolder('/kaggle/input/handwritten-dataset/test', transform=val_transform)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [6]:
num_classes = len(train_dataset.classes)
# ====== CNN Model ======
class MathCNN(nn.Module):
    def __init__(self, num_classes=13):
        super(MathCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()

        # Automatically compute flatten size
        self._to_linear = None
        self._dummy_forward()
        
        self.fc1 = nn.Linear(self._to_linear, 128)
        self.fc2 = nn.Linear(128, num_classes)
    
    def _dummy_forward(self):
        x = torch.randn(1,1,28,28)
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        self._to_linear = x.numel()  # total features after conv+pool
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MathCNN(num_classes=num_classes).to(device)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# ====== Training Loop ======
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")
    
    # ====== Validation ======
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    val_acc = 100 * val_correct / val_total
    print(f"Validation Accuracy: {val_acc:.2f}%\n")

Epoch 1/15, Loss: 0.5598, Train Acc: 82.60%
Validation Accuracy: 95.88%

Epoch 2/15, Loss: 0.2315, Train Acc: 92.74%
Validation Accuracy: 96.65%

Epoch 3/15, Loss: 0.1622, Train Acc: 95.03%
Validation Accuracy: 98.44%

Epoch 4/15, Loss: 0.1317, Train Acc: 95.83%
Validation Accuracy: 98.00%

Epoch 5/15, Loss: 0.1090, Train Acc: 96.58%
Validation Accuracy: 98.52%

Epoch 6/15, Loss: 0.0942, Train Acc: 97.02%
Validation Accuracy: 98.95%

Epoch 7/15, Loss: 0.0837, Train Acc: 97.37%
Validation Accuracy: 99.34%

Epoch 8/15, Loss: 0.0777, Train Acc: 97.65%
Validation Accuracy: 99.19%

Epoch 9/15, Loss: 0.0683, Train Acc: 97.87%
Validation Accuracy: 99.14%

Epoch 10/15, Loss: 0.0630, Train Acc: 98.02%
Validation Accuracy: 98.97%

Epoch 11/15, Loss: 0.0604, Train Acc: 98.13%
Validation Accuracy: 99.21%

Epoch 12/15, Loss: 0.0517, Train Acc: 98.32%
Validation Accuracy: 99.16%

Epoch 13/15, Loss: 0.0543, Train Acc: 98.29%
Validation Accuracy: 99.42%

Epoch 14/15, Loss: 0.0503, Train Acc: 98.43%
Va

In [9]:
torch.save(model.state_dict(), "math_cnn.pth")

In [10]:
from IPython.display import FileLink
FileLink("math_cnn.pth")


/kaggle/working/math_cnn.pth